In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output
import utils
from utils import keras_baselilne
from utils import resnet

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
#train_final = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train_final.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")

#train = pd.concat([train,train_final])

train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)  
print("done!")

done!


In [3]:


# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


#X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
#                    , X_angle_train, y_train, random_state=123, train_size=0.75)



In [4]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

reload(utils)
model = utils.keras_baselilne_batch()
model.summary()

utils.py:229: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
  img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
utils.py:231: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
  img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
utils.py:236: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
  img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
utils.py:238: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
  img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
utils.py:243: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
utils.py:245: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  img_1 = Conv2D(64,3,3, activation=

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 75, 75, 3)    12          X_1[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 73, 73, 16)   448         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 73, 73, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
file_path = "/mnt/extDisk/courses/data/statoil/data/models/.keras_baseline_full.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model = utils.keras_baselilne_batch()
model.fit([X_train, X_angle_train], y_train, epochs=25
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=32
         , callbacks=callbacks)



Train on 602 samples, validate on 200 samples
Epoch 1/25
602/602 [==============================] - 5s 8ms/step - loss: 1.0326 - acc: 0.5797 - val_loss: 1.2052 - val_acc: 0.5500
Epoch 2/25
602/602 [==============================] - 1s 1ms/step - loss: 0.7904 - acc: 0.6561 - val_loss: 1.5862 - val_acc: 0.5500
Epoch 3/25
602/602 [==============================] - 1s 1ms/step - loss: 0.5789 - acc: 0.7043 - val_loss: 0.8900 - val_acc: 0.5500
Epoch 4/25
602/602 [==============================] - 1s 1ms/step - loss: 0.5844 - acc: 0.7309 - val_loss: 1.3608 - val_acc: 0.5550
Epoch 5/25
602/602 [==============================] - 1s 1ms/step - loss: 0.5858 - acc: 0.7442 - val_loss: 1.3922 - val_acc: 0.5900
Epoch 6/25
602/602 [==============================] - 1s 1ms/step - loss: 0.5562 - acc: 0.7658 - val_loss: 0.8030 - val_acc: 0.5800
Epoch 7/25
602/602 [==============================] - 1s 1ms/step - loss: 0.4530 - acc: 0.7708 - val_loss: 0.4549 - val_acc: 0.7400
Epoch 8/25
602/602 [==========

In [6]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
602/602 [==============================] - 0s 517us/step
[0.22361459793442506, 0.91029902946117314]
####################
watch list evaluate:
200/200 [==============================] - 0s 251us/step
[0.27510794997215271, 0.88499999046325684]


In [8]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 2s 195us/step


In [9]:
prediction = np.clip(prediction,0.05,0.95)

In [10]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.050000
1,4023181e,0.050000
2,b20200e4,0.050000
3,e7f018bb,0.898577
4,4371c8c3,0.050000
5,a8d9b1fd,0.323968
6,29e7727e,0.630151
7,92a51ffb,0.950000
8,c769ac97,0.050000
9,aee0547d,0.050000


In [11]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/train/keras_baseline_full.csv", index=False)

In [17]:
X_train.shape

(602, 75, 75, 3)